In [ ]:
pip install imblearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls /content/drive/MyDrive/MIT805_Exam/

Census2022sample_F18.csv  Census2022sample_F19.csv  Census2022sample_F21.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

In [ ]:
data_1 = pd.read_csv('/content/drive/MyDrive/MIT805_Exam/Census2022sample_F21.csv')
data_2 = pd.read_csv('/content/drive/MyDrive/MIT805_Exam/Census2022sample_F18.csv')
data_3 = pd.read_csv('/content/drive/MyDrive/MIT805_Exam/Census2022sample_F19.csv')

In [ ]:
# Perform the join on the 'QID' column
merged_data = data_1.merge(data_2, on='QID', how='inner').merge(data_3, on='QID', how='inner')

In [ ]:
merged_data

,QID,PID,P02_SEX,P03_YEAR,P03_MONTH,AGE_GROUP,P04_AGE,P05_RELATION,P06_MARITAL_ST,P07A_POP_GROUP,...,H12_DVD_PLAYER,H12_MOTOR_CAR,H12_TELEVISION,H12_RADIO,H12_LANDLINE,H12_CELLPHONE,H13_INTERNET_ACCESS,A4_ADULT_HUNGER,A5_CHILD_HUNGER,HH_WGT
0,10000003,10000003001,2,1977,8,9,44,1,6,1,...,2,2,2,2,2,1,2,1,1,15.584189
1,10000003,10000003002,2,2004,8,4,17,3,6,1,...,2,2,2,2,2,1,2,1,1,15.584189
2,10000005,10000005001,2,1980,99,9,41,1,1,1,...,2,1,1,1,2,1,1,1,2,15.584189
3,10000005,10000005002,1,1978,99,9,43,2,1,1,...,2,1,1,1,2,1,1,1,2,15.584189
4,10000005,10000005003,1,2008,99,3,13,3,6,1,...,2,1,1,1,2,1,1,1,2,15.584189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4230787,91338294,91338294004,2,2003,10,4,18,3,6,1,...,2,2,1,2,2,1,9,1,1,11.836005
4230788,91338294,91338294005,2,2013,9,2,8,3,8,1,...,2,2,1,2,2,1,9,1,1,11.836005
4230789,91338294,91338294006,1,2017,6,1,4,10,8,1,...,2,2,1,2,2,1,9,1,1,11.836005
4230790,91338294,91338294007,1,2021,7,1,0,10,8,1,...,2,2,1,2,2,1,9,1,1,11.836005


In [ ]:
# List of columns to select
columns_to_select = [ 'Province', "District", 'Municipality',
    'QID', 'PID', 'P02_SEX', 'P04_AGE',
    "P17A_SEEING", "P17B_HEARING", "P17C_COMMUNICATION", "P17D_WALKING", "P17E_REMEMBERING", "P17F_SELF_CARE",
    "P17A_DEVMEDEYEGLAS", "P17B_DEVMEDHEARINGAID", "P17C_DEVMEWALKINGSTICK", "P17E_PROSTHESIS", "P17D_DEVMEDWHEELCHAIR",
    "DISABILITY_STATUS",
    "A4_ADULT_HUNGER", "A5_CHILD_HUNGER",
    "H05_WATERPIPED", "H06_WATERSOURCE", "H07A_WATERSUPPLY",
    "H08_TOILET",
    "H09_ENERGY_COOKING", "H10_ENERGY_LIGHTING"
]

# Select columns from the dataframe
selected_df = merged_data[columns_to_select]


In [ ]:
selected_df.to_csv('/content/drive/MyDrive/MIT805_Exam/Final_Data/Census_Data_Analysis.csv')

In [ ]:
merged_data = data_1.merge(data_2, on='QID', how='inner').merge(data_3, on='QID', how='inner')
df = merged_data

# Create binary target for Adult Hunger
df['Adult_Hunger'] = df['A4_ADULT_HUNGER'].apply(lambda x: 1 if x == 5 else 0)

# Create binary target for Child Hunger (optional)
df['Child_Hunger'] = df['A5_CHILD_HUNGER'].apply(lambda x: 1 if x == 5 else 0)


In [ ]:
print("Adult Hunger distribution:")
print(df['Adult_Hunger'].value_counts())

print("\nChild Hunger distribution:")
print(df['Child_Hunger'].value_counts())


Adult Hunger distribution:
Adult_Hunger
0    4170643
1      60149
Name: count, dtype: int64

Child Hunger distribution:
Child_Hunger
0    4177506
1      53286
Name: count, dtype: int64


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import warnings

# Encoding Province as a numerical feature
df['Province_encoded'] = LabelEncoder().fit_transform(df['Province'])

# Define predictors and target
X = df[['DERH_HSIZE',
    'P02_SEX', 'P04_AGE',
    "P17A_SEEING", "P17B_HEARING", "P17C_COMMUNICATION", "P17D_WALKING", "P17E_REMEMBERING", "P17F_SELF_CARE",
    "P17A_DEVMEDEYEGLAS", "P17B_DEVMEDHEARINGAID", "P17C_DEVMEWALKINGSTICK", "P17E_PROSTHESIS", "P17D_DEVMEDWHEELCHAIR",
    "DISABILITY_STATUS",
    "H05_WATERPIPED", "H06_WATERSOURCE", "H07A_WATERSUPPLY",
    "H08_TOILET", 'P20_EDUINST', 'Province_encoded']]
y = df['Adult_Hunger']  # Use Child_Hunger if analyzing child hunger

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Apply SMOTE to balance the training set
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Standardize the features
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

# Standardize the features
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

# Train logistic regression model with balanced class weights
model = LogisticRegression(class_weight='balanced')
model.fit(X_train_balanced, y_train_balanced)


# Make predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Accuracy Score
print("\nAccuracy Score:", accuracy_score(y_test, y_pred))

# AUC-ROC Score
auc_score = roc_auc_score(y_test, y_proba)
print("\nAUC-ROC Score:", auc_score)

# Display model coefficients
print("\nModel coefficients:")
feature_names = ['DERH_HSIZE',
    'P02_SEX', 'P04_AGE',
    "P17A_SEEING", "P17B_HEARING", "P17C_COMMUNICATION", "P17D_WALKING", "P17E_REMEMBERING", "P17F_SELF_CARE",
    "P17A_DEVMEDEYEGLAS", "P17B_DEVMEDHEARINGAID", "P17C_DEVMEWALKINGSTICK", "P17E_PROSTHESIS", "P17D_DEVMEDWHEELCHAIR",
    "DISABILITY_STATUS",
    "H05_WATERPIPED", "H06_WATERSOURCE", "H07A_WATERSUPPLY",
    "H08_TOILET", 'P20_EDUINST', 'Province_encoded']
for coef, feature in zip(model.coef_[0], feature_names):
    print(f"{feature}: {coef}")


Confusion Matrix:
[[691997 559196]
 [  6725  11320]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.55      0.71   1251193
           1       0.02      0.63      0.04     18045

    accuracy                           0.55   1269238
   macro avg       0.51      0.59      0.37   1269238
weighted avg       0.98      0.55      0.70   1269238


Accuracy Score: 0.5541253886189982

AUC-ROC Score: 0.6412877946406266

Model coefficients:
DERH_HSIZE: 0.021768842610635185
P02_SEX: -0.30580688613605594
P04_AGE: -0.0585627479396813
P17A_SEEING: -0.8294238308055925
P17B_HEARING: -0.36177919411845033
P17C_COMMUNICATION: 0.20725086377137902
P17D_WALKING: -0.4122505881501662
P17E_REMEMBERING: -0.03798302334285112
P17F_SELF_CARE: 0.3655671583017502
P17A_DEVMEDEYEGLAS: 0.04723030836395134
P17B_DEVMEDHEARINGAID: 0.7216461986529991
P17C_DEVMEWALKINGSTICK: -0.33793151698610024
P17E_PROSTHESIS: -0.5903637124254152
P17D_DEVMEDWHEELCHAIR: -0.317027